# Applied Data Science Capstone: Week 3 Assignment
### Clustering Toronto's Neighborhoods



## Table of contents

I. Creating the Neighborhoods dataframe  
II. Geocoding  
III. k-means Clustering
  

In [1]:
# import libraries

import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
import folium # map rendering library
from sklearn.cluster import KMeans
import requests

#import json
import matplotlib.cm as cm
import matplotlib.colors as colors

pd.set_option('display.max_columns', None)

## I. Creating the Neighborhoods dataframe

In [2]:
# To get the same result as the assignment's instructions, I retrieved a previous revision of the wikipedia page

url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"    # unused
old_url = "https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=945633050"

df1 = pd.read_html(old_url)[0]    # store table in old_url in df1
df1

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
...,...,...,...
282,M8Z,Etobicoke,Mimico NW
283,M8Z,Etobicoke,The Queensway West
284,M8Z,Etobicoke,Royal York South West
285,M8Z,Etobicoke,South of Bloor


In [3]:
#Rename 'Postcode' to 'PostalCode' and drop rows with 'Not assigned' Boroughs

df1.columns = ['PostalCode', 'Borough', 'Neighborhood']             # rename columns
df1.drop(df1[df1.Borough == 'Not assigned'].index, inplace=True)    # drop rows with no Boroughs
df1.reset_index(drop=True, inplace=True)                            # reset index
df1

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor
...,...,...,...
205,M8Z,Etobicoke,Kingsway Park South West
206,M8Z,Etobicoke,Mimico NW
207,M8Z,Etobicoke,The Queensway West
208,M8Z,Etobicoke,Royal York South West


In [4]:
# Double check if there are any 'Not assigned' Neighborhoods

df1.loc[df1['Neighborhood'] == "Not assigned"]    # should return no rows

,PostalCode,Borough,Neighborhood


In [5]:
# Join Neighborhoods with identical PostalCode

df2 = df1.groupby(['PostalCode', 'Borough'])['Neighborhood'].apply(', '.join).reset_index()
df2.head(5)

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [6]:
# Get the number of rows of df2

df2.shape

(103, 3)

## II. Geocoding

In [7]:
# Using the provided csv in lieu of geolocator API

df_geo = pd.read_csv("Geospatial_Coordinates.csv")
df_geo.head(5)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [8]:
# Append latitude and longitudes in df_geo to df2 by PostalCode

df3 = df2.join(df_geo.set_index('Postal Code'), on='PostalCode')
df3

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv...",43.688905,-79.554724
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ...",43.739416,-79.588437


In [9]:
# Get Toronto's coordinates from geolocator for Folium's map centering

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode('Toronto, Canada')
print("Latitude:", location.latitude, "Longitude:", location.longitude)

Latitude: 43.6534817 Longitude: -79.3839347


In [10]:
# Just in case geolocator is unresponsive, saving the values to these variables. Call these variables instead when Toronto's latitude/longitude is needed.

toronto_latitude = 43.6534817
toronto_longitude = -79.3839347

In [11]:
# Build folium map of Toronto and mark neighborhoods according to dataframe df3

map_toronto = folium.Map(location=[toronto_latitude, toronto_longitude], zoom_start=11)

for lat, lng, label in zip(df3['Latitude'], df3['Longitude'], df3['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  

map_toronto

  
## III. k-means Clustering

In [12]:
# Define and call function to retrieve venues from foursquare. Lifted from previous lab.

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            "L35L4SIWUDE1OFSVAJ2SITX2KW5MN4VZJHZXGDVWZ5QX4HAC", 
            "LQ54QTZQ4JK2UNZQOUJVR23ZQENZ3FH1IEX1KYYZ2DQVF2YI", 
            "20180604", 
            lat, 
            lng, 
            "500", 
            "100")
            
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)


toronto_venues = getNearbyVenues(names = df3['Neighborhood'], latitudes = df3['Latitude'], longitudes = df3['Longitude'])

Rouge, Malvern
Highland Creek, Rouge Hill, Port Union
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Clarks Corners, Sullivan, Tam O'Shanter
Agincourt North, L'Amoreaux East, Milliken, Steeles East
L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
Silver Hills, York Mills
Newtonbrook, Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park, Don Mills South
Bathurst Manor, Downsview North, Wilson Heights
Northwood Park, York University
CFB Toronto, Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens, Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West, 

In [13]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Malvern",43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,SEBS Engineering Inc. (Sustainable Energy and ...,43.782371,-79.156820,Construction & Landscaping
3,"Guildwood, Morningside, West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Big Bite Burrito,43.766299,-79.190720,Mexican Restaurant


toronto_venues is a list of venues.

One way to convert them into numbers is to convert each venue row into occurrences of its Venue Category.

We will count Venue Category occurrences via one hot encoding.

In [14]:
# One hot encoding to dataframe toronto_onehot

toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")        # one hot encoding

# The Neighborhood column is somewhere in the middle of this long list of columns. To move Neighborhood into left-most column:

toronto_onehot.drop('Neighborhood', axis = 1, inplace = True)                                        # drop neighborhood for re-add next line
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood']                                      # add neighborhood column back to dataframe
toronto_onehot = toronto_onehot[[toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])]    # move neighborhood column to the first column

# Combine rows with identical Neighborhoods

toronto_grouped = toronto_onehot.groupby('Neighborhood').sum().reset_index()
toronto_grouped.head()

,Neighborhood,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Butcher,Cable Car,Cafeteria,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Performing Arts Venue,Pet Store,Pharmacy,Piano Bar,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Soccer Field,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Summer Camp,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide, King, Richmond",0,0,0,0,0,0,3,0,0,1,0,1,0,0,0,0,0,0,2,0,2,0,0,0,0,0,0,0,0,0,0,0,0,2,0,1,2,0,0,0,1,1,1,0,0,0,0,0,5,0,0,0,0,0,0,0,0,3,0,8,0,0,0,0,0,1,0,0,2,0,0,2,0,0,0,1,0,0,

In [25]:
# Define and call function 

num_top_venues = 10


# Create columns according to number of top venues

columns = ['Neighborhood']
indicators = ['st', 'nd', 'rd']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# Create a new dataframe

neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']


# Call function

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Restaurant,American Restaurant,Thai Restaurant,Hotel,Gym,Clothing Store,Deli / Bodega,Pizza Place
1,Agincourt,Lounge,Skating Rink,Latin American Restaurant,Breakfast Spot,Electronics Store,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Park,Playground,Coffee Shop,Eastern European Restaurant,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Grocery Store,Fried Chicken Joint,Pharmacy,Pizza Place,Discount Store,Sandwich Place,Liquor Store,Beer Store,Japanese Restaurant,Fast Food Restaurant
4,"Alderwood, Long Branch",Pizza Place,Skating Rink,Gym,Pharmacy,Coffee Shop,Pub,Sandwich Place,Pool,Donut Shop,Discount Store


Setup k-means cluster for 5 clusters

In [26]:
# Run k-means for 4 clusters

kclusters = 4
toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', axis = 1)                                       # drop Neighborhood column so it doesn't get clustered
kmeans = KMeans(n_clusters = kclusters, random_state=0).fit(toronto_grouped_clustering)                           # run k-means
kmeans.labels_[0:10]
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)                                           # append cluster labels to dataframe

# Merge kmeans-labeled dataframe 'neighborhoods_venues_sorted' with toronto_data to add latitude/longitude for each neighborhood

toronto_merged = df3.copy()                                                                                       # clone new dataframe from df3
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')    # join dataframes
toronto_merged = toronto_merged[toronto_merged['Cluster Labels'].notna()]                                         # remove rows neighborhoods with no cluster labels
toronto_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,1.0,Fast Food Restaurant,Yoga Studio,Discount Store,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,1.0,Construction & Landscaping,Bar,Yoga Studio,Empanada Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,1.0,Rental Car Location,Bank,Electronics Store,Medical Center,Intersection,Mexican Restaurant,Breakfast Spot,Cupcake Shop,Curling Ice,Farmers Market
3,M1G,Scarborough,Woburn,43.770992,-79.216917,1.0,Coffee Shop,Korean Restaurant,Yoga Studio,Electronics Store,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Ethiopian Restaurant
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,1.0,Bank,Bakery,Hakka Restaurant,Fried Chicken Joint,Gas Station,Thai Restaurant,Athletics & Sports,Caribbean Restaurant,Donut Shop,Distribution Center


In [49]:
# create map

map_clusters = folium.Map(location=[toronto_latitude, toronto_longitude], zoom_start=12)

x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)],
        fill=True,
        fill_color=rainbow[int(cluster)],
        fill_opacity=0.7).add_to(map_clusters)

map_clusters

**Observations from map above and from dataframes of the clusters**

1. Locations of clusters
 * Purples (cluster 0) are mostly found close to the city center
 * Teals (cluster 1) are marked for almost all the city's suburban areas.
 * Greens (cluster 2) are found only in the city center
 * Reds (cluster 3) are found in 3 locations, 2 in the city center and 1 in the suburbs

2. Why are there 3 types of clusters in the city center, i.e. Purples, Greens, and Reds?
 * Greens (cluster 2) appear to be upper-scale commercial venues, without establishments like Supermarkets and Sandwish Places that are otherwise found in Purples (cluster 0)
 * Reds (cluster 3) appear to be clusters with predominantly clothing stores; see df_cluster3 below.
 
3. Teals (cluster 1) comprise commercial venues that are typical in suburban areas.


In [50]:
# Purple, cluster 0

df_cluster0 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]
df_cluster0

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,Willowdale South,0.0,Ramen Restaurant,Sandwich Place,Coffee Shop,Pizza Place,Café,Sushi Restaurant,Japanese Restaurant,Lounge,Fast Food Restaurant,Plaza
38,Leaside,0.0,Sporting Goods Shop,Coffee Shop,Burger Joint,Bank,Furniture / Home Store,Sports Bar,Beer Store,Liquor Store,Bike Shop,Supermarket
41,"The Danforth West, Riverdale",0.0,Greek Restaurant,Italian Restaurant,Coffee Shop,Bookstore,Furniture / Home Store,Ice Cream Shop,Yoga Studio,Bubble Tea Shop,Spa,Japanese Restaurant
43,Studio District,0.0,Café,Coffee Shop,Gastropub,Bakery,American Restaurant,Brewery,Latin American Restaurant,Italian Restaurant,Fish Market,Bookstore
47,Davisville,0.0,Pizza Place,Dessert Shop,Sandwich Place,Gym,Italian Restaurant,Café,Coffee Shop,Sushi Restaurant,Brewery,Restaurant
51,"Cabbagetown, St. James Town",0.0,Coffee Shop,Park,Café,Italian Restaurant,Bakery,Pizza Place,Chinese Restaurant,Pub,Restaurant,Gift Shop
52,Church and Wellesley,0.0,Sushi Restaurant,Coffee Shop,Japanese Restaurant,Restaurant,Yoga Studio,Hotel,Mediterranean Restaurant,Men's Store,Gastropub,Pub
53,Harbourfront,0.0,Coffee Shop,Pub,Park,Bakery,Breakfast Spot,Theater,Café,Restaurant,Chocolate Shop,Distribution Center
55,St. James Town,0.0,Café,Coffee Shop,Gastropub,Italian Restaurant,Hotel,American Restaurant,Cocktail Bar,Creperie,Lingerie Store,Gym
56,Berczy Park,0.0,Coffee Shop,Seafood Restaurant,Cocktail Bar,Beer Bar,Bakery,Farmers Market,Italian Restaurant,Restaurant,Cheese Shop,Café


In [51]:
# Teal, cluster 1

df_cluster1 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]
df_cluster1

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Rouge, Malvern",1.0,Fast Food Restaurant,Yoga Studio,Discount Store,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant
1,"Highland Creek, Rouge Hill, Port Union",1.0,Construction & Landscaping,Bar,Yoga Studio,Empanada Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant
2,"Guildwood, Morningside, West Hill",1.0,Rental Car Location,Bank,Electronics Store,Medical Center,Intersection,Mexican Restaurant,Breakfast Spot,Cupcake Shop,Curling Ice,Farmers Market
3,Woburn,1.0,Coffee Shop,Korean Restaurant,Yoga Studio,Electronics Store,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Ethiopian Restaurant
4,Cedarbrae,1.0,Bank,Bakery,Hakka Restaurant,Fried Chicken Joint,Gas Station,Thai Restaurant,Athletics & Sports,Caribbean Restaurant,Donut Shop,Distribution Center
...,...,...,...,...,...,...,...,...,...,...,...,...
97,"Emery, Humberlea",1.0,Baseball Field,Food Service,Yoga Studio,Electronics Store,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Empanada Restaurant
99,Westmount,1.0,Pizza Place,Intersection,Chinese Restaurant,Coffee Shop,Discount Store,Sandwich Place,Yoga Studio,Drugstore,Dog Run,Doner Restaurant
100,"Kingsview Village, Martin Grove Gardens, Richv...",1.0,Park,Pizza Place,Sandwich Place,Mobile Phone Shop,Ethiopian Restaurant,Event Space,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Diner
101,"Albion Gardens, Beaumond Heights, Humbergate, ...",1.0,Grocery Store,Fried Chicken Joint,Pharmacy,Pizza Place,Discount Store,Sandwich Place,Liquor Store,Beer Store,Japanese Restaurant,Fast Food Restaurant


In [52]:
# Green, cluster 2

df_cluster2 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]
df_cluster2

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
57,Central Bay Street,2.0,Coffee Shop,Italian Restaurant,Café,Sandwich Place,Fried Chicken Joint,Japanese Restaurant,Bubble Tea Shop,Salad Place,Burger Joint,Ice Cream Shop
59,"Harbourfront East, Toronto Islands, Union Station",2.0,Coffee Shop,Aquarium,Hotel,Café,Restaurant,Italian Restaurant,Brewery,Scenic Lookout,Fried Chicken Joint,Sporting Goods Shop
60,"Design Exchange, Toronto Dominion Centre",2.0,Coffee Shop,Hotel,Café,Restaurant,Salad Place,Japanese Restaurant,American Restaurant,Seafood Restaurant,Tea Room,Deli / Bodega
61,"Commerce Court, Victoria Hotel",2.0,Coffee Shop,Café,Restaurant,Hotel,American Restaurant,Gym,Italian Restaurant,Seafood Restaurant,Deli / Bodega,Japanese Restaurant
69,Stn A PO Boxes 25 The Esplanade,2.0,Coffee Shop,Italian Restaurant,Seafood Restaurant,Café,Restaurant,Beer Bar,Japanese Restaurant,Hotel,Cheese Shop,Park
70,"First Canadian Place, Underground city",2.0,Coffee Shop,Café,Restaurant,Hotel,Japanese Restaurant,Bar,Steakhouse,American Restaurant,Seafood Restaurant,Gym


In [53]:
# Red, cluster 3

df_cluster3 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]
df_cluster3

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,"Fairview, Henry Farm, Oriole",3.0,Clothing Store,Coffee Shop,Fast Food Restaurant,Restaurant,Women's Store,Sporting Goods Shop,Bakery,Bank,Bus Station,Japanese Restaurant
54,"Ryerson, Garden District",3.0,Clothing Store,Coffee Shop,Café,Middle Eastern Restaurant,Cosmetics Shop,Japanese Restaurant,Restaurant,Bubble Tea Shop,Italian Restaurant,Ramen Restaurant
58,"Adelaide, King, Richmond",3.0,Coffee Shop,Café,Restaurant,American Restaurant,Thai Restaurant,Hotel,Gym,Clothing Store,Deli / Bodega,Pizza Place
